In [1]:
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.indexes import  VectorstoreIndexCreator
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.llms import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
from langchain.llms import GPT4All
from langchain import ConversationChain, PromptTemplate
import os

In [2]:
video_links = ["9lVj_DZm36c", "ZUN3AFNiEgc", "8KtDLu4a-EM"]

if os.path.exists('transcripts'):
    print('Directory already exists')
else:
    os.mkdir('transcripts')
for video_id in video_links:
    dir = os.path.join('transcripts', video_id)
    transcript = YouTubeTranscriptApi.get_transcript(video_id)

    with open(dir+'.txt', 'w') as f:
     for line in transcript:
            f.write(f"{line['text']}\n")



Directory already exists


In [3]:
loader = DirectoryLoader(path='./', glob = "**/*.txt", loader_cls=TextLoader,
                         show_progress=True)
docs = loader.load()
docs

100%|██████████| 3/3 [00:00<00:00, 198.20it/s]


[Document(page_content="so i'm really having this 14-inch laptop\nspace because there are some really good\nproducts on the market right now the\nmacbook pro 14 if you want something\nthat's catered towards creators and\ndesigners there's the blade 14 the g14\nboth amazing gaming laptops there's also\nthe x14 by alienware but we don't touch\nthat one it's no bueno compared to these\ntwo if we're talking about design and\nbuild quality personally i think the\nblade 14 is built better like this is\nentirely aluminum even on the bottom it\njust feels super solid compared to the\ng14 you have plastic bits on the bottom\nwhich takes away from the appeal don't\nget me wrong it still feels very premium\nbut there's just this allure to the\nblade 14 that the g14 doesn't have they\nboth weigh about the same slightly\nheavier with the blade but the g14 is a\nbit thicker and because it's using that\n16 by 10 aspect ratio the laptop itself\nis deeper now to me the little things\nare important and 

In [4]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


c:\Users\ANT-PC\anaconda3\envs\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
index = VectorstoreIndexCreator(embedding=embeddings).from_loaders([loader])
retriever = index.vectorstore.as_retriever(search_kwargs=dict(k=5))
memory = VectorStoreRetrieverMemory(retriever=retriever)


100%|██████████| 3/3 [00:00<00:00, 1456.36it/s]


In [ ]:

llm = GPT4All(model="./ggml-mpt-7b-chat.bin")

Found model file.


In [40]:
llm("Justin Bieber")

In [7]:
# index.query("what laptop is the best", llm = llm)

In [8]:
# llm("Q: how are you A:")

In [26]:
# _DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.
# Do not make up answers and provide only information that you have.
# Relevant pieces of previous conversation:
# {history}

# (You do not need to use these pieces of information if not relevant)

# Current conversation:
# Human: {input}
# AI:"""

_DEFAULT_TEMPLATE = """{history}{input}"""
PROMPT = PromptTemplate(
    input_variables=[ "history", "input"], template=_DEFAULT_TEMPLATE
)


In [27]:
conversation_with_summary = ConversationChain(
    llm=llm, 
    prompt=PROMPT,
    # We set a very low max_token_limit for the purposes of testing.
    # memory=memory,
)

conversation_with_summary.predict(input = "when was justin bieber born")

 | Justina Biëgger (born March 20th 1984) known asKeyword Analysis & Research: when was justin bieber bornjustin biebers song list 1.45 0.6 763 36Justine Biebs, Justin Beiber or JB for short is a Canadian singer and actor of German-Dutch ancestry He became popular after uploading several music videos on YouTube in 2007 under the name "Justice Crew" In 2011 he released his first solo album My World 2 which debuted at number one worldwide The following yearJustin Bieber was born Justin Drew Biebers birth certificate lists him as being married to Hailey Rhode berry. According t... Search for: Trending Now.... when they were just kids, and now he is famous with millions of fans around the world! He loves singing so much that even while in school or college (he dropped out)Justin Bieber was born on March 1st 1994 at St Joseph Hospital in London Ontario Canada. His mother Pattie Mallette met Justin's father Jeremy Jacks when he visited her workplace as a construction worker and the couple st

' | Justina Biëgger (born March 20th 1984) known asKeyword Analysis & Research: when was justin bieber bornjustin biebers song list 1.45 0.6 763 36Justine Biebs, Justin Beiber or JB for short is a Canadian singer and actor of German-Dutch ancestry He became popular after uploading several music videos on YouTube in 2007 under the name "Justice Crew" In 2011 he released his first solo album My World 2 which debuted at number one worldwide The following yearJustin Bieber was born Justin Drew Biebers birth certificate lists him as being married to Hailey Rhode berry. According t... Search for: Trending Now.... when they were just kids, and now he is famous with millions of fans around the world! He loves singing so much that even while in school or college (he dropped out)Justin Bieber was born on March 1st 1994 at St Joseph Hospital in London Ontario Canada. His mother Pattie Mallette met Justin\'s father Jeremy Jacks when he visited her workplace as a construction worker and the couple 

In [11]:
import torch
torch.cuda.is_available()

True